# Quickstart

To best understand how NutritionAI can give your agents super food-nutrition powers, let's build an agent that can find that information via Passio NutritionAI.

## Define tools

We first need to create [the Passio NutritionAI tool](/docs/integrations/tools/passio_nutrition_ai).

### [Passio Nutrition AI](/docs/integrations/tools/passio_nutrition_ai)

We have a built-in tool in LangChain to easily use Passio NutritionAI to find food nutrition facts.
Note that this requires an API key - they have a free tier.

Once you create your API key, you will need to export that as:

```bash
export NUTRITIONAI_SUBSCRIPTION_KEY="..."
```

... or provide it to your Python environment via some other means such as the `dotenv` package.  You an also explicitly control the key via constructor calls.

In [10]:
from dotenv import load_dotenv
from langchain_core.utils import get_from_env

load_dotenv()

nutritionai_subscription_key = get_from_env(
    "nutritionai_subscription_key", "NUTRITIONAI_SUBSCRIPTION_KEY"
)

In [11]:
from langchain_community.tools.passio_nutrition_ai import NutritionAI
from langchain_community.utilities.passio_nutrition_ai import NutritionAIAPI

In [12]:
nutritionai_search = NutritionAI(api_wrapper=NutritionAIAPI())

In [13]:
nutritionai_search.invoke("chicken tikka masala")

{'results': [{'type': 'reference',
   'displayName': 'Chicken Tikka Masala',
   'stemmedDisplayName': '',
   'shortName': '',
   'longName': 'Chicken Tikka Masala',
   'scoredName': 'Chicken Tikka Masala',
   'score': 1,
   'displayNameScore': 1,
   'brandName': '',
   'iconId': '',
   'labelId': '',
   'synonymId': '',
   'recipeId': '',
   'referenceId': 'openfood0085239308349',
   'resultId': 'openfood0085239308349',
   'nutritionPreview': {'portion': {'weight': {'unit': 'g', 'value': 100},
     'name': '',
     'quantity': 1,
     'suggestedQuantity': [1]},
    'calories': 142.8571014404297}},
  {'type': 'reference',
   'displayName': 'Chicken Tikka Masala',
   'stemmedDisplayName': '',
   'shortName': '',
   'longName': 'Chicken Tikka Masala',
   'scoredName': 'Chicken Tikka Masala',
   'score': 0.95,
   'displayNameScore': 0.95,
   'brandName': 'Wells Enterprises, Inc.',
   'iconId': '',
   'labelId': '',
   'synonymId': '',
   'recipeId': '',
   'referenceId': 'openfood539000301

In [14]:
nutritionai_search.invoke("Schnuck Markets sliced pepper jack cheese")

{'results': [{'type': 'reference',
   'displayName': 'Schnuck Markets, Sliced Colby Cheese',
   'stemmedDisplayName': '',
   'shortName': '',
   'longName': 'Schnuck Markets, Sliced Colby Cheese',
   'scoredName': 'Schnuck Markets, Sliced Colby Cheese',
   'score': 0.8260839471482273,
   'displayNameScore': 0.8260839471482273,
   'brandName': 'Schnuck Markets, Inc.',
   'iconId': '',
   'labelId': '',
   'synonymId': '',
   'recipeId': '',
   'referenceId': '1603211583771',
   'resultId': '1603211583771',
   'nutritionPreview': {'portion': {'weight': {'unit': 'g', 'value': 23},
     'name': 'slice',
     'quantity': 1},
    'calories': 89.93000030517578}},
  {'type': 'reference',
   'displayName': 'Deli Sliced Pepper Jack Cheese',
   'stemmedDisplayName': '',
   'shortName': '',
   'longName': 'Deli Sliced Pepper Jack Cheese',
   'scoredName': 'Deli Sliced Pepper Jack Cheese',
   'score': 0.7994458979182494,
   'displayNameScore': 0.7994458979182494,
   'brandName': "MEMBER'S MARK",
  

### Tools

Now that we have the tool, we can create a list of tools that we will use downstream.

In [15]:
tools = [nutritionai_search]

## Create the agent

Now that we have defined the tools, we can create the agent. We will be using an OpenAI Functions agent - for more information on this type of agent, as well as other options, see [this guide](/docs/modules/agents/agent_types/)

First, we choose the LLM we want to be guiding the agent.

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

Next, we choose the prompt we want to use to guide the agent.

In [17]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Now, we can initalize the agent with the LLM, the prompt, and the tools. The agent is responsible for taking in input and deciding what actions to take. Crucially, the Agent does not execute those actions - that is done by the AgentExecutor (next step). For more information about how to think about these components, see our [conceptual guide](/docs/modules/agents/concepts)

In [18]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

Finally, we combine the agent (the brains) with the tools inside the AgentExecutor (which will repeatedly call the agent and execute tools). For more information about how to think about these components, see our [conceptual guide](/docs/modules/agents/concepts)

In [19]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## Run the agent

We can now run the agent on a few queries! Note that for now, these are all **stateless** queries (it won't remember previous interactions).

In [20]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [21]:
agent_executor.invoke({"input": "how many calories are in a slice pepperoni pizza?"})



> Entering new AgentExecutor chain...

Invoking: `nutritionai_advanced_search` with `{'query': 'slice of pepperoni pizza'}`


{'results': [{'type': 'synonym', 'displayName': 'Fast Food, Pizza Chain, 14" Pizza, Pepperoni Topping, Regular Crust', 'stemmedDisplayName': '', 'shortName': 'slice of pepperoni pizza', 'longName': 'Fast Food, Pizza Chain, 14" Pizza, Pepperoni Topping, Regular Crust', 'scoredName': 'Slice Of Pepperoni Pizza', 'score': 1, 'displayNameScore': 0.6534380115251439, 'brandName': '', 'iconId': '1003594', 'labelId': '1f1bf95c-3256-11ed-920f-7e029295fe1f', 'synonymId': '1f1bf95f-3256-11ed-920f-7e029295fe1f', 'recipeId': '', 'referenceId': '1603211199398', 'resultId': '1603211199398', 'nutritionPreview': {'portion': {'weight': {'unit': 'g', 'value': 111}, 'name': 'slice', 'quantity': 1}, 'calories': 313.0199890136719}}, {'type': 'synonym', 'displayName': 'Little Caesars 14" Original Round Pepperoni Pizza, Regular Crust', 'stemmedDisplayName': '', 'shortName': 'slice of 

{'input': 'how many calories are in a slice pepperoni pizza?',
 'output': 'A slice of pepperoni pizza typically contains around 245-350 calories, depending on the specific brand and type of crust.'}

If we want to keep track of these messages automatically, we can wrap this in a RunnableWithMessageHistory. For more information on how to use this, see [this guide](/docs/expression_language/how_to/message_history)

In [22]:
agent_executor.invoke(
    {"input": "I had bacon and eggs for breakfast.  How many calories is that?"}
)



> Entering new AgentExecutor chain...

Invoking: `nutritionai_advanced_search` with `{'query': 'bacon and eggs'}`


{'results': [{'type': 'recipe', 'displayName': 'English Muffin With Egg, Bacon And Cheese', 'stemmedDisplayName': '', 'shortName': 'English Muffin with Egg, Bacon and Cheese', 'longName': '', 'scoredName': 'Bacon And Egg Mcmuffin', 'score': 0.9336296056884292, 'displayNameScore': 0.7684418158923452, 'brandName': '', 'iconId': 'PRE0398', 'labelId': 'fccdbf14-cc85-11ea-965a-ab82c246b581', 'synonymId': '75da9f7e-7dd6-11eb-8ccb-c3fcc89180a9', 'recipeId': '68d57155-da31-11ec-b94d-ded9c6f70628', 'referenceId': '', 'resultId': '68d57155-da31-11ec-b94d-ded9c6f70628', 'nutritionPreview': {'portion': {'weight': {'unit': 'g', 'value': 148.1}, 'name': 'serving', 'quantity': 1, 'suggestedQuantity': [1]}, 'calories': 331.40802001953125}}, {'type': 'recipe', 'displayName': 'Puff Pastry Filled With Bacon And Egg', 'stemmedDisplayName': '', 'shortName': 'puff pastry filled with bacon an

{'input': 'I had bacon and eggs for breakfast.  How many calories is that?',
 'output': 'The bacon and eggs you had for breakfast contain approximately 331 calories.'}

In [23]:
agent_executor.invoke(
    {
        "input": "I had sliced pepper jack cheese for a snack.  How much protein did I have?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `nutritionai_advanced_search` with `{'query': 'sliced pepper jack cheese'}`


{'results': [{'type': 'reference', 'displayName': 'Sliced Pepper Jack Cheese', 'stemmedDisplayName': '', 'shortName': '', 'longName': 'Sliced Pepper Jack Cheese', 'scoredName': 'Sliced Pepper Jack Cheese', 'score': 1, 'displayNameScore': 1, 'brandName': 'Tops Markets, LLC', 'iconId': '', 'labelId': '', 'synonymId': '', 'recipeId': '', 'referenceId': '1603211277022', 'resultId': '1603211277022', 'nutritionPreview': {'portion': {'weight': {'unit': 'g', 'value': 23}, 'name': 'slice', 'quantity': 1}, 'calories': 89.93000030517578}}, {'type': 'reference', 'displayName': 'Sliced Pepper Jack Cheese', 'stemmedDisplayName': '', 'shortName': '', 'longName': 'Sliced Pepper Jack Cheese', 'scoredName': 'Sliced Pepper Jack Cheese', 'score': 0.95, 'displayNameScore': 0.95, 'brandName': '', 'iconId': '', 'labelId': '', 'synonymId': '', 'recipeId': '', 'referenceId': 'openfoo

{'input': 'I had sliced pepper jack cheese for a snack.  How much protein did I have?',
 'output': 'The sliced pepper jack cheese you had contains approximately 5 grams of protein per 1 slice (23g).'}

In [24]:
agent_executor.invoke(
    {
        "input": "I had sliced colby cheese for a snack. Give me calories for this Schnuck Markets product."
    }
)



> Entering new AgentExecutor chain...

Invoking: `nutritionai_advanced_search` with `{'query': 'Schnuck Markets sliced colby cheese'}`


{'results': [{'type': 'reference', 'displayName': 'Schnuck Markets, Sliced Colby Cheese', 'stemmedDisplayName': '', 'shortName': '', 'longName': 'Schnuck Markets, Sliced Colby Cheese', 'scoredName': 'Schnuck Markets, Sliced Colby Cheese', 'score': 0.9361508951406649, 'displayNameScore': 0.9361508951406649, 'brandName': 'Schnuck Markets, Inc.', 'iconId': '', 'labelId': '', 'synonymId': '', 'recipeId': '', 'referenceId': '1603211583771', 'resultId': '1603211583771', 'nutritionPreview': {'portion': {'weight': {'unit': 'g', 'value': 23}, 'name': 'slice', 'quantity': 1}, 'calories': 89.93000030517578}}, {'type': 'synonym', 'displayName': 'Kroger Colby Jack Sliced Cheese', 'stemmedDisplayName': '', 'shortName': 'sliced colby cheese', 'longName': 'Kroger Colby Jack Sliced Cheese', 'scoredName': 'Sliced Colby Cheese', 'score': 0.881964219181897, 'displayNam

{'input': 'I had sliced colby cheese for a snack. Give me calories for this Schnuck Markets product.',
 'output': 'The Schnuck Markets sliced colby cheese contains approximately 89.93 calories per slice (23g).'}

In [25]:
agent_executor.invoke(
    {
        "input": "I had chicken tikka masala for dinner.  how much calories, protein, and fat did I have with default quantity?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `nutritionai_advanced_search` with `{'query': 'chicken tikka masala'}`


{'results': [{'type': 'reference', 'displayName': 'Chicken Tikka Masala', 'stemmedDisplayName': '', 'shortName': '', 'longName': 'Chicken Tikka Masala', 'scoredName': 'Chicken Tikka Masala', 'score': 1, 'displayNameScore': 1, 'brandName': '', 'iconId': '', 'labelId': '', 'synonymId': '', 'recipeId': '', 'referenceId': 'openfood0085239308349', 'resultId': 'openfood0085239308349', 'nutritionPreview': {'portion': {'weight': {'unit': 'g', 'value': 100}, 'name': '', 'quantity': 1, 'suggestedQuantity': [1]}, 'calories': 142.8571014404297}}, {'type': 'reference', 'displayName': 'Chicken Tikka Masala', 'stemmedDisplayName': '', 'shortName': '', 'longName': 'Chicken Tikka Masala', 'scoredName': 'Chicken Tikka Masala', 'score': 0.95, 'displayNameScore': 0.95, 'brandName': 'Wells Enterprises, Inc.', 'iconId': '', 'labelId': '', 'synonymId': '', 'recipeId': '', 'referenceId':

{'input': 'I had chicken tikka masala for dinner.  how much calories, protein, and fat did I have with default quantity?',
 'output': 'I found nutrition information for Chicken Tikka Masala. Here are the details for a 140g serving:\n\n- Calories: 179.99\n- Protein: 14.7g\n- Fat: 10.5g\n\nPlease note that these values are approximate and may vary based on the specific recipe or brand of Chicken Tikka Masala you consumed.'}

## Conclusion

That's a wrap! In this quick start we covered how to create a simple agent that is able to incorporate food-nutrition information into its answers. Agents are a complex topic, and there's lot to learn!